In [0]:
# ! pip install beautifulsoup4
from bs4 import BeautifulSoup
from urllib.request import urlopen
url = "https://webscraper.io/test-sites/tables"
html_code= urlopen(url).read().decode()
soup = BeautifulSoup(html_code,"html.parser")
table = soup.find("table")
names_list = table.find_all('th')
tables = [title.text.strip() for title in names_list]
import pandas as pd
df= pd.DataFrame(columns = tables)
column_data = table.findAll('tr')
for row in column_data[1:]:
    row_data = row.find_all('td')
    individual_row_data = [data.text.strip() for data in row_data]

    length = len(df)
    df.loc[length] = individual_row_data

df.to_csv("NameList",index = False)

In [0]:
dbutils.fs.ls('file:/databricks/driver')

Out[5]: [FileInfo(path='file:/databricks/driver/conf/', name='conf/', size=4096, modificationTime=1710926178381),
 FileInfo(path='file:/databricks/driver/preload_class.lst', name='preload_class.lst', size=1306936, modificationTime=1710926179741),
 FileInfo(path='file:/databricks/driver/azure/', name='azure/', size=4096, modificationTime=1710926179629),
 FileInfo(path='file:/databricks/driver/hadoop_accessed_config.lst', name='hadoop_accessed_config.lst', size=2755, modificationTime=1710926179661),
 FileInfo(path='file:/databricks/driver/ganglia/', name='ganglia/', size=4096, modificationTime=1710929703008),
 FileInfo(path='file:/databricks/driver/logs/', name='logs/', size=4096, modificationTime=1710929568104),
 FileInfo(path='file:/databricks/driver/eventlogs/', name='eventlogs/', size=4096, modificationTime=1710926432309),
 FileInfo(path='file:/databricks/driver/.cache', name='.cache', size=205, modificationTime=1710928862436),
 FileInfo(path='file:/databricks/driver/NameList', name=

In [0]:
dbutils.fs.mv('file:/databricks/driver/NameList', '/NameList.csv', recurse=True)

Out[6]: True

In [0]:
pp = spark.read.csv('/NameList.csv')
display(pp)

_c0,_c1,_c2,_c3
#,First Name,Last Name,Username
1,Mark,Otto,@mdo
2,Jacob,Thornton,@fat
3,Larry,the Bird,@twitter


In [0]:
%pip install snowflake
%pip install snowflake-connector-python

import pandas as pd
from pyspark.sql import SparkSession
import snowflake.connector

# Snowflake connection parameters
account = 'My Account'
user = 'My User Name'
password = 'My Password'
warehouse = 'My Warehouse Name'
database = 'My Database Name'
schema = 'My Schema Name'
table_name = 'My Table Name'  # Table name without '.csv' suffix

# Snowflake URL
snowflake_url = ''

# Load CSV into a Spark DataFrame
# file_path = "/games_table1.csv/part-00000-tid-7662509018983859286-e9725b8e-b65d-492e-a84f-bce0f1026892-129-1-c000.csv"
file_path = "/NameList.csv"
spark = SparkSession.builder.appName("SnowflakeIntegration").getOrCreate()
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Snowflake connection
conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema,
    sfURL=snowflake_url  # Include Snowflake URL
)

# Create cursor
cur = conn.cursor()

# Replace spaces and dots in table name
table_name_sanitized = table_name.replace(' ', '_').replace('.', '_')

# Create table in Snowflake with dynamically generated column definitions
column_definitions = ', '.join([f'"{col}" STRING' for col in df.columns])
cur.execute(f'CREATE OR REPLACE TABLE "{schema}"."{table_name_sanitized}" ({column_definitions})')

# Write DataFrame to Snowflake table
df.write.format("snowflake") \
    .option("sfURL", snowflake_url) \
    .option("sfUser", user) \
    .option("sfPassword", password) \
    .option("sfDatabase", database) \
    .option("sfSchema", schema) \
    .option("sfWarehouse", warehouse) \
    .option("dbtable", table_name_sanitized) \
    .mode("overwrite") \
    .save()

# Commit the transaction
conn.commit()

# Close cursor and connection
cur.close()
conn.close()


Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
